In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/CM/roi_patches")
print (os.path.abspath('.')) #获取当前工作目录路径
print(os.listdir('.'))

/content/gdrive/MyDrive/CM/roi_patches
['test_X_2_18.npy', 'test_Y_2_18.npy', 'train_X.npy', 'train_Y.npy', 'test_X.npy', 'test_Y.npy', 'point_X_3_8.npy', 'point_Y_3_8.npy', 'all_patches.npy', 'all_labels.npy', 'all_train_X.npy', 'all_train_Y.npy', 'all_val_X.npy', 'all_val_Y.npy', 'all_test_X.npy', 'all_test_Y.npy', 'all_patches_point.npy', 'all_labels_point.npy', 'all_patches_10.npy', 'all_labels_10.npy', 'zhangbei_X.npy', 'zhangbei_Y.npy', 'zhangbei_unlabeled_X.npy', 'zhangbei_baseline_X.npy', 'zhangbei_baseline_Y.npy', 'zb_test_Y_r=0.npy', 'zb_test_X_r=0.npy', 'zb_train_Y_r=0.npy', 'zb_train_X_r=0.npy', 'zb_test_Y_r=5.npy', 'zb_train_X_r=5.npy', 'zb_train_Y_r=5.npy', 'zb_val_Y_r=5.npy', 'zb_val_X_r=5.npy', 'zb_test_X_r=5.npy', 'zhangbei_delete_bad_X.npy', 'zhangbei_delete_bad_Y.npy']


In [ ]:
from __future__ import print_function, division
import numpy as np
import tifffile as tiff
from PIL import Image
import codecs
import os
import cv2
import scipy.io as sio
from keras.utils.np_utils import to_categorical
from scipy.cluster.vq import whiten
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import random
from collections import Counter

In [ ]:
#load npy
origin_dataset_X = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/train_x.npy')
origin_dataset_Y = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/train_y.npy')

In [ ]:
origin_dataset_X.shape

(1900, 15, 15, 12)

In [ ]:
save_path = "/content/gdrive/MyDrive/CM/RSL/RSL_dataset"

In [ ]:
factor = 0.8#抽取比例
def divide_dataset(dataset,p):
  #打乱某类数据集
  index = np.random.permutation(dataset.shape[0])
  dataset = dataset[index]
  # print(dataset.shape)
  length = dataset.shape[0]
  # print(length)
  part1 = int(length*p)
  part2 = length - part1
  dataset1 = dataset[0:part1,:]
  dataset2 = dataset[part1:,:]
  print('该类划分结果:'+str(dataset1.shape)+str(dataset2.shape))
  return dataset1,dataset2

In [ ]:
def get_dataset(dataset_X,dataset_Y,p): 
  X1 = []
  Y1 = []
  X2 = []
  Y2 = []
  flag = 0
  for i in range(1,8):#范围根据类别需要修改
    classes = i 
    print('当前类别为：'+ str(classes))
    total_dataset = dataset_X[dataset_Y == classes]
    train_set,test_set = divide_dataset(total_dataset,p)
    l1 = train_set.shape[0]
    l2 = test_set.shape[0]
    
    if flag == 0 :
      train_set = train_set.tolist()
      test_set = test_set.tolist()
      X1.append(train_set)
      X2.append(test_set)
      Y1 = Y1 + [classes for j in range(0,l1)]
      Y2 = Y2 + [classes for k in range(0,l2)]
      X1 = np.asarray(X1[0],dtype=float)
      X2 = np.asarray(X2[0],dtype=float)
      flag = 1
    else:
      train_set = np.asarray(train_set,dtype = float)
      test_set = np.asarray(test_set,dtype = float)
      X1 = np.concatenate((X1, train_set),axis=0)
      X2 = np.concatenate((X2, test_set),axis=0)
      Y1 = Y1 + [classes for j in range(0,l1)]
      Y2 = Y2 + [classes for k in range(0,l2)]
      print('当前训练集测试集大小:'+str(X1.shape)+str(X2.shape))
  Y1 = np.asarray(Y1,dtype = int)   
  Y2 = np.asarray(Y2,dtype = int)
  return X1,Y1,X2,Y2

In [ ]:
X_train,Y_train,X_test,Y_test = get_dataset(origin_dataset_X,origin_dataset_Y,factor)

当前类别为：1
该类划分结果:(240, 15, 15, 12)(60, 15, 15, 12)
当前类别为：2
该类划分结果:(240, 15, 15, 12)(60, 15, 15, 12)
当前训练集测试集大小:(480, 15, 15, 12)(120, 15, 15, 12)
当前类别为：3
该类划分结果:(240, 15, 15, 12)(60, 15, 15, 12)
当前训练集测试集大小:(720, 15, 15, 12)(180, 15, 15, 12)
当前类别为：4
该类划分结果:(160, 15, 15, 12)(40, 15, 15, 12)
当前训练集测试集大小:(880, 15, 15, 12)(220, 15, 15, 12)
当前类别为：5
该类划分结果:(240, 15, 15, 12)(60, 15, 15, 12)
当前训练集测试集大小:(1120, 15, 15, 12)(280, 15, 15, 12)
当前类别为：6
该类划分结果:(240, 15, 15, 12)(60, 15, 15, 12)
当前训练集测试集大小:(1360, 15, 15, 12)(340, 15, 15, 12)
当前类别为：7
该类划分结果:(160, 15, 15, 12)(40, 15, 15, 12)
当前训练集测试集大小:(1520, 15, 15, 12)(380, 15, 15, 12)


In [ ]:
X_train1,Y_train1,X_val,Y_val = get_dataset(X_train,Y_train,0.8)

当前类别为：1
该类划分结果:(1008, 11, 11, 7)(252, 11, 11, 7)
当前类别为：2
该类划分结果:(1008, 11, 11, 7)(252, 11, 11, 7)
当前训练集测试集大小:(2016, 11, 11, 7)(504, 11, 11, 7)


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)

(2016, 11, 11, 7)
(1080, 11, 11, 7)
(504, 11, 11, 7)


In [ ]:
Counter(Y_test)

In [ ]:
np.save(save_path+'/train_x.npy', X_train)
np.save(save_path+'/train_y.npy', Y_train)
# np.save(save_path+'/area_val_X_r=5.npy', X_val)
# np.save(save_path+'/area_val_Y_r=5.npy', Y_val)
np.save(save_path+'/val_x.npy', X_test)
np.save(save_path+'/val_y.npy', Y_test)